# **Web Scrapping para extraer la data conprimida en los archivos parquet de la pagina TLC NYC**
# ☝

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importo librerias necesarias
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

# Yellow Taxis data dictionary

| Field Name              | Description                                               |
|-------------------------|-----------------------------------------------------------|
| VendorID                | A code indicating the TPEP provider that provided the record. 1= Creative Mobile Technologies, LLC; 2= VeriFone Inc. |
| tpep_pickup_datetime    | The date and time when the meter was engaged.            |
| tpep_dropoff_datetime   | The date and time when the meter was disengaged.          |
| Passenger_count         | The number of passengers in the vehicle. This is a driver-entered value. |
| Trip_distance           | The elapsed trip distance in miles reported by the taximeter. |
| PULocationID            | TLC Taxi Zone in which the taximeter was engaged.         |
| DOLocationID            | TLC Taxi Zone in which the taximeter was disengaged.       |
| RateCodeID              | The final rate code in effect at the end of the trip.     |
|                         | 1= Standard rate                                        |
|                         | 2= JFK                                                 |
|                         | 3= Newark                                              |
|                         | 4= Nassau or Westchester                                |
|                         | 5= Negotiated fare                                     |
|                         | 6= Group ride                                          |
| Store_and_fwd_flag      | This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka “store and forward,” because the vehicle did not have a connection to the server. Y= store and forward trip, N= not a store and forward trip |
| Payment_type            | A numeric code signifying how the passenger paid for the trip. 1= Credit card, 2= Cash, 3= No charge, 4= Dispute, 5= Unknown, 6= Voided trip |
| Fare_amount             | The time-and-distance fare calculated by the meter.      |
| Extra                   | Miscellaneous extras and surcharges. Currently, this only includes the $0.50 and $1 rush hour and overnight charges. |
| MTA_tax                 | $0.50 MTA tax that is automatically triggered based on the metered rate in use. |
| Improvement_surcharge   | $0.30 improvement surcharge assessed trips at the flag drop. The improvement surcharge began being levied in 2015. |
| Tip_amount              | Tip amount – This field is automatically populated for credit card tips. Cash tips are not included. |
| Tolls_amount            | Total amount of all tolls paid in trip.                  |
| Total_amount            | The total amount charged to passengers. Does not include cash tips. |
| Congestion_Surcharge    | Total amount collected in trip for NYS congestion surcharge. |
| Airport_fee             | $1.25 for pick up only at LaGuardia and John F. Kennedy Airports. |


# Green Taxis data dictionary
| Field Name              | Description                                               |
|-------------------------|-----------------------------------------------------------|
| VendorID                | A code indicating the LPEP provider that provided the record. 1= Creative Mobile Technologies, LLC; 2= VeriFone Inc. |
| lpep_pickup_datetime    | The date and time when the meter was engaged.            |
| lpep_dropoff_datetime   | The date and time when the meter was disengaged.          |
| Passenger_count         | The number of passengers in the vehicle. This is a driver-entered value. |
| Trip_distance           | The elapsed trip distance in miles reported by the taximeter. |
| PULocationID            | TLC Taxi Zone in which the taximeter was engaged.         |
| DOLocationID            | TLC Taxi Zone in which the taximeter was disengaged.       |
| RateCodeID              | The final rate code in effect at the end of the trip.     |
|                         | 1= Standard rate                                        |
|                         | 2= JFK                                                 |
|                         | 3= Newark                                              |
|                         | 4= Nassau or Westchester                                |
|                         | 5= Negotiated fare                                     |
|                         | 6= Group ride                                          |
| Store_and_fwd_flag      | This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka “store and forward,” because the vehicle did not have a connection to the server. Y= store and forward trip, N= not a store and forward trip |
| Payment_type            | A numeric code signifying how the passenger paid for the trip. 1= Credit card, 2= Cash, 3= No charge, 4= Dispute, 5= Unknown, 6= Voided trip |
| Fare_amount             | The time-and-distance fare calculated by the meter.      |
| Extra                   | Miscellaneous extras and surcharges. Currently, this only includes the $0.50 and $1 rush hour and overnight charges. |
| MTA_tax                 | $0.50 MTA tax that is automatically triggered based on the metered rate in use. |
| Improvement_surcharge   | $0.30 improvement surcharge assessed on hailed trips at the flag drop. The improvement surcharge began being levied in 2015. |
| Tip_amount              | Tip amount – This field is automatically populated for credit card tips. Cash tips are not included. |
| Tolls_amount            | Total amount of all tolls paid in trip.                  |
| Total_amount            | The total amount charged to passengers. Does not include cash tips. |
| Trip_type               | A code indicating whether the trip was a street-hail or a dispatch that is automatically assigned based on the metered rate in use but can be altered by the driver. 1= Street-hail, 2= Dispatch |


# For Hire Vehicles data dictionary
| Field Name                 | Description                                                                                                  |
|----------------------------|--------------------------------------------------------------------------------------------------------------|
| Dispatching_base_num       | The TLC Base License Number of the base that dispatched the trip                                         |
| Pickup_datetime            | The date and time of the trip pick-up                                                                      |
| DropOff_datetime           | The date and time of the trip dropoff                                                                      |
| PULocationID               | TLC Taxi Zone in which the trip began                                                                      |
| DOLocationID               | TLC Taxi Zone in which the trip ended                                                                      |
| SR_Flag                    | Indicates if the trip was a part of a shared ride chain offered by a High Volume FHV company (e.g. Uber Pool, Lyft Line). For shared trips, the value is 1. For non-shared rides, this field is null. NOTE: For most High Volume FHV companies, only shared rides that were requested AND matched to another shared-ride request over the course of the journey are flagged. However, Lyft (base license numbers B025


# High Volume For Hire Vehicles data dictionary
| Field Name             | Description                                                                                                           |
|------------------------|-----------------------------------------------------------------------------------------------------------------------|
| Hvfhs_license_num      | The TLC license number of the HVFHS base or business. As of September 2019, the HVFHS licensees are the following: HV0002: Juno, HV0003: Uber, HV0004: Via, HV0005: Lyft |
| Dispatching_base_num   | The TLC Base License Number of the base that dispatched the trip                                                    |
| Pickup_datetime        | The date and time of the trip pick-up                                                                              |
| DropOff_datetime       | The date and time of the trip drop-off                                                                             |
| PULocationID           | TLC Taxi Zone in which the trip began                                                                              |
| DOLocationID           | TLC Taxi Zone in which the trip ended                                                                              |
| originating_base_num   | Base number of the base that received the original trip request                                                   |
| request_datetime       | Date/time when passenger requested to be picked up                                                                |
| on_scene_datetime      | Date/time when driver arrived at the pick-up location (Accessible Vehicles-only)                                    |
| trip_miles             | Total miles for passenger trip                                                                                     |
| trip_time              | Total time in seconds for passenger trip                                                                           |
| base_passenger_fare    | Base passenger fare before tolls, tips, taxes, and fees                                                          |
| tolls                  | Total amount of all tolls paid in trip                                                                            |
| bcf                    | Total amount collected in trip for Black Car Fund                                                                 |
| sales_tax              | Total amount collected in trip for NYS sales tax                                                                  |
| congestion_surcharge   | Total amount collected in trip for NYS congestion surcharge                                                       |
| airport_fee            | $2.50 for both drop off and pick up at LaGuardia, Newark, and John F. Kennedy airports                            |
| tips                   | Total amount of tips received from passenger                                                                      |
| driver_pay             | Total driver pay (not including tolls or tips and net of commission, surcharges, or taxes)                        |
| shared_request_flag    | Did the passenger agree to a shared/pooled ride, regardless of whether they were matched? (Y/N)                   |
| shared_match_flag      | Did the passenger share the vehicle with another passenger who booked separately at any point during the trip? (Y/N) |
| access_a_ride_flag     | Was the trip administered on behalf of the Metropolitan Transportation Authority (MTA)? (Y/N)                      |
| wav_request_flag       | Did the passenger request a wheelchair-accessible vehicle (WAV)? (Y/N)                                            |
| wav_match_flag         | Did the trip occur in a wheelchair-accessible vehicle (WAV)? (Y/N)                                                |


### URL de la pagina de TLC (Taxis & Limousine Commission)

In [ ]:
url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

### Hago un GET para obtener la respuesta del servidor de la pagina y asi obtener su estructura HTML

In [ ]:
response = requests.get(url)

### Utilizando *BeautifulSoup* extraigo el contenido de la pagina en un objeto llamado 'soup'

In [ ]:
soup = BeautifulSoup(response.content, "html.parser") # Contenido de la pagina
parquet_files = [] # Creo una lista para alojar todos los links que sean de los archivos parquet

In [ ]:
soup

<!DOCTYPE html>

<!--[if lt IE 7]><html class="no-js lt-ie9 lt-ie8 lt-ie7"><![endif]--><!--[if IE 7]><html class="no-js lt-ie9 lt-ie8 ie7"><![endif]--><!--[if IE 8]><html class="no-js lt-ie9"><![endif]--><!--[if gt IE 8]><!--><html class="no-js"><!--<![endif]--><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>TLC Trip Record Data - TLC</title>
<!--
					ls:begin[stylesheet]
				-->
<link href="/iwov-resources/fixed-layout/3-Row Simple.css" rel="stylesheet" type="text/css"/>
<!--
					ls:end[stylesheet]
				-->
<!--
					ls:begin[meta-keywords]
				-->
<meta content="" name="keywords"/>
<!--
					ls:end[meta-keywords]
				-->
<!--
					ls:begin[meta-description]
				-->
<meta content="" name="description"/>
<!--
					ls:end[meta-description]
				-->
<!--
					ls:begin[meta-vpath]
				-->
<meta content="" name="vpath"/>
<!--
					ls:end[meta-vpath]
				-->
<!--
					ls:begin[meta-page-locale-name]
				-->
<meta content="" name="page-locale-name"/>
<!--
		

### Busco en la estructura de la pagina todos los elementos que sean href, es decir, links, y que terminen con .parquet para poder diferenciar aquellos archivos que estamo buscando

In [ ]:
# Hago una filtración de los archivos que pertenecen solamente a los años que nos importan
selected_years = ['2017', '2018', '2019']
years = [2017,2018,2019]

for link in soup.find_all('a',href=True): # Busco todos los elementos que sean links
  if link['href'].endswith('.parquet'): # Si el elemento termina en .parquet, añado el elemento a la lista parquet_files
    if any(year in link['href'] for year in selected_years):
      parquet_files.append(link['href'])
print(f'{len(parquet_files)} links parquet se cargaron')

357 links parquet se cargaron


In [ ]:
numero_de_elementos_parquet= len(parquet_files)
numero_de_elementos_parquet

161

### Como tenemos cuatro tipos de transportes *(Yellow Taxis, Green Taxis, For Hire Vehicles y High Volume For Hire Vehicles)*, creo una lista para cada uno para asi poder organizar los links en función a su categoria

In [ ]:
yellow_taxis = []
green_taxis = []
for_hire_vehicles = []
high_volume_for_hire_vehicles = []

### Itero la lista de *parquet_files* y en función a las palabras que contenga cada link será organizado en la lista de su categoria/tipo correspondiente.

In [ ]:
for link in parquet_files:
  if 'yellow_tripdata' in link: # Todos los links que contengan 'yellow_tripdata' en su texto
    yellow_taxis.append(link)
  elif 'green_tripdata' in link: # Todos los links que contengan 'green_tripdata' en su texto
    green_taxis.append(link)
  elif 'fhv_tripdata' in link: # Todos los links que contengan 'fhv_tripdata' en su texto
    for_hire_vehicles.append(link)
  elif 'fhvhv_tripdata' in link: # Todos los links que contengan 'fhvhv_tripdata' en su texto
    high_volume_for_hire_vehicles.append(link)


In [ ]:
yellow_taxis = sorted(yellow_taxis, reverse=True)
green_taxis = sorted(green_taxis, reverse=True)
for_hire_vehicles = sorted(for_hire_vehicles, reverse=True)
high_volume_for_hire_vehicles = sorted(high_volume_for_hire_vehicles, reverse=True)

In [ ]:
def download_parquet_links(links_list, taxi_color):
  with open(f'drive/MyDrive/Proyecto Final Henry/Uriel/parquet_links_taxis/{taxi_color}_parquet_links.json', 'w') as json_file:
    json.dump(links_list, json_file)
  print(f'Se descargo un archivo llamado "{taxi_color}_parquet_links.json"')

In [ ]:
download_parquet_links(yellow_taxis, 'yellow')
download_parquet_links(green_taxis, 'green')
download_parquet_links(for_hire_vehicles, 'grey')
download_parquet_links(high_volume_for_hire_vehicles, 'black')

Se descargo un archivo llamado "yellow_parquet_links.json"
Se descargo un archivo llamado "green_parquet_links.json"
Se descargo un archivo llamado "grey_parquet_links.json"
Se descargo un archivo llamado "black_parquet_links.json"


In [ ]:
pd.read_parquet(yellow_taxis[-1]).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6405008 entries, 0 to 6405007
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

### Creo los dataframes para cada link y luego hago una concatenación de los links de cada tipo, de manera que tendriamos toda la info de cada tipo de transporte en un solo dataframe

In [ ]:
manhattan_zones = [4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 103, 104, 105, 107, 113, 114, 116, 120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]

In [ ]:
def transformation_green_taxis(transport_list):
  dataframes = []
  for parquet_file in transport_list:
    df = pd.read_parquet(parquet_file)
    # Selecciono solamente los registros que pertenecen a viajes del distrito de Manhattan
    df = df[df.PULocationID.isin(manhattan_zones) | df.DOLocationID.isin(manhattan_zones)].reset_index(drop=True)
    dataframes.append(df)

  # Ahora concateno todos esos dataframes en uno solo
  full_df = pd.concat(dataframes, ignore_index=True)
  full_df['VendorID'] = full_df['VendorID'].replace(1, 'Creative Mobile Technologies, LLC').replace(2, 'VeriFone Inc.')
  full_df['RatecodeID'] = full_df.RatecodeID.replace(1, 'Standard rate').replace(2, 'JFK').replace(3, 'Newark').replace(4, 'Nassau or Westchester').replace(5, 'Negotiated fare').replace(6, 'Group ride')
  full_df['store_and_fwd_flag'] = full_df['store_and_fwd_flag'].replace('Y', 'store and forward trip').replace('N', 'not a store and forward trip')
  full_df['payment_type'] = full_df['payment_type'].replace(1, 'Credit card').replace(2, 'Cash').replace(3, 'No charge').replace(4,'Dispute').replace(5,'Unknown').replace(6,'Voided trip')


  return full_df

In [ ]:
pd.read_parquet(green_taxis[-1])

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2019-12-18 15:52:30,2019-12-18 15:54:39,N,1.0,264,264,5.0,0.00,3.50,0.50,0.5,0.01,0.0,None,0.3,4.81,1.0,1.0,0.00
1,2,2020-01-01 00:45:58,2020-01-01 00:56:39,N,5.0,66,65,2.0,1.28,20.00,0.00,0.0,4.06,0.0,None,0.3,24.36,1.0,2.0,0.00
2,2,2020-01-01 00:41:38,2020-01-01 00:52:49,N,1.0,181,228,1.0,2.47,10.50,0.50,0.5,3.54,0.0,None,0.3,15.34,1.0,1.0,0.00
3,1,2020-01-01 00:52:46,2020-01-01 01:14:21,N,1.0,129,263,2.0,6.30,21.00,3.25,0.5,0.00,0.0,None,0.3,25.05,2.0,1.0,2.75
4,1,2020-01-01 00:19:57,2020-01-01 00:30:56,N,1.0,210,150,1.0,2.30,10.00,0.50,0.5,0.00,0.0,None,0.3,11.30,1.0,1.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447765,2,2020-01-31 23:29:00,2020-01-31 23:47:00,None,NaN,167,32,NaN,4.58,23.21,2.75,0.0,0.00,0.0,None,0.3,26.26,NaN,NaN,NaN
447766,2,2020-01-31 23:57:00,2020-02-01 00:23:00,None,NaN,81,69,NaN,6.55,27.27,2.75,0.0,0.00,0.0,None,0.3,30.32,NaN,NaN,NaN
447767,2,2020-01-31 23:57:00,2020-02-01 00:10:00,None,NaN,244,241,NaN,3.34,25.95,2.75,0.0,0.00,0.0,None,0.3,29.00,NaN,NaN,NaN
447768,2,2020-01-31 23:27:00,2020-02-01 00:04:00,None,NaN,68,17,NaN,8.92,30.39,2.75,0.0,0.00,0.0,None,0.3,33.44,NaN,NaN,NaN


---

## 👇 <mark>**Para descargar un fragmento de los TAXIS AMARILLOS**</mark> 👇

*(solo 200 registros)*

In [ ]:
pd.read_parquet(yellow_taxis[0])[:200].to_csv('taxis_amarillos.csv',sep=',')

## 👇 <mark>**Para descargar un fragmento de los TAXIS VERDES**</mark> 👇

*(solo 200 registros)*

In [ ]:
pd.read_parquet(green_taxis[0])[:200].to_csv('taxis_verdes.csv',sep=',')

## 👇 <mark>**Para descargar un fragmento de los TAXIS GRISES**</mark> 👇


*(solo 200 registros)*

In [ ]:
pd.read_parquet(for_hire_vehicles[0])[:200].to_csv('taxis_grises.csv',sep=',')

## 👇 <mark>**Para descargar un fragmento de los TAXIS NEGROS**</mark> 👇

*(solo 200 registros)*

In [ ]:
pd.read_parquet(high_volume_for_hire_vehicles[0])[:200].to_csv('taxis_negros.csv',sep=',')

KeyboardInterrupt: ignored

---

In [ ]:
def green_taxis_transformation(files_list):
  dataframes = []
  for parquet_file in files_list:
    df = pd.read_parquet(parquet_file)
    # Selecciono solamente los registros que pertenecen a viajes del distrito de Manhattan
    df = df[df.PULocationID.isin(manhattan_zones) | df.DOLocationID.isin(manhattan_zones)].reset_index(drop=True)
    dataframes.append(df)
  # Ahora concateno todos esos dataframes en uno solo
  full_df = pd.concat(dataframes, ignore_index=True)

  return full_df

def for_hire_transformation(files_list):
  pass

def high_volume_transformation(files_list):
  pass

### Realizo el proceso para cada tipo de transporte
###### *(El codigo esta comentado ya que requiere de mucho poder de computo)*

In [ ]:
# yellow_taxis_df = full_dataframe(yellow_taxis)
# green_taxis_df = full_dataframe(green_taxis)
# fhv_df = full_dataframe(for_hire_vehicles)
# fhvhv_df = full_dataframe(high_volume_for_hire_vehicles)

NameError: ignored

In [ ]:
green_taxis_df = concating_green_data(green_taxis)

NameError: ignored

### Demostración con el dataframe de taxis verdes que no es muy pesado

In [ ]:
green_taxis_df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2023-04-01 00:01:46,2023-04-01 00:11:42,N,1.0,75,42,1.0,2.62,13.50,1.00,0.5,3.00,0.00,NaN,1.0,19.00,1.0,1.0,0.00
1,2,2023-04-01 00:21:32,2023-04-01 00:53:16,N,1.0,166,107,1.0,6.51,33.80,1.00,0.5,7.81,0.00,NaN,1.0,46.86,1.0,1.0,2.75
2,2,2023-04-01 00:03:14,2023-04-01 00:16:54,N,1.0,74,238,1.0,2.59,15.60,1.00,0.5,2.50,0.00,NaN,1.0,20.60,1.0,1.0,0.00
3,1,2023-04-01 00:36:23,2023-04-01 00:57:57,N,1.0,112,140,1.0,7.50,32.40,3.75,1.5,5.00,0.00,NaN,1.0,42.65,1.0,1.0,2.75
4,2,2023-04-01 00:20:50,2023-04-01 00:45:07,N,1.0,66,158,1.0,4.26,24.70,1.00,0.5,3.00,0.00,NaN,1.0,32.95,1.0,1.0,2.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975344,2,2020-01-31 23:22:00,2020-01-31 23:51:00,None,NaN,41,49,NaN,12.44,35.72,2.75,0.0,0.00,6.12,None,0.3,44.89,NaN,NaN,NaN
1975345,2,2020-01-31 23:22:00,2020-02-01 00:16:00,None,NaN,41,187,NaN,26.17,57.84,2.75,0.0,0.00,30.11,None,0.3,91.00,NaN,NaN,NaN
1975346,2,2020-01-31 23:13:00,2020-01-31 23:35:00,None,NaN,94,75,NaN,7.47,35.79,2.75,0.0,0.00,0.00,None,0.3,38.84,NaN,NaN,NaN
1975347,2,2020-01-31 23:57:00,2020-02-01 00:10:00,None,NaN,244,241,NaN,3.34,25.95,2.75,0.0,0.00,0.00,None,0.3,29.00,NaN,NaN,NaN


In [ ]:
green_taxis_df.to_csv('taxis_green_dos.csv')

---

In [ ]:
df_yellow_taxis = pd.read_parquet(yellow_taxis[0])
df_green_taxis = pd.read_parquet(green_taxis[0])
df_fh_vehicles = pd.read_parquet(for_hire_vehicles[0])
df_hvfh_vehicles = pd.read_parquet(high_volume_for_hire_vehicles[0])

## Comparación de columnas entre los cuatro tipos de transportes

In [ ]:
df_yellow_taxis.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')

In [ ]:
df_green_taxis.columns

Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge'],
      dtype='object')

In [ ]:
df_fh_vehicles.columns

Index(['dispatching_base_num', 'pickup_datetime', 'dropOff_datetime',
       'PUlocationID', 'DOlocationID', 'SR_Flag', 'Affiliated_base_number'],
      dtype='object')

In [ ]:
df_hvfh_vehicles.columns

Index(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag'],
      dtype='object')

In [ ]:
df_yellow_taxis = pd.read_parquet(yellow_taxis[0])
df_yellow_taxis.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-04-01 00:14:49,2023-04-01 00:45:01,2.0,4.90,1.0,N,48,223,1,28.9,3.5,0.5,6.0,0.00,1.0,39.9,2.5,0.00
1,2,2023-04-01 00:00:24,2023-04-01 00:56:19,1.0,21.89,2.0,N,132,43,2,70.0,0.0,0.5,0.0,6.55,1.0,81.8,2.5,1.25
2,1,2023-04-01 00:03:50,2023-04-01 00:14:42,2.0,1.30,1.0,N,148,113,1,11.4,3.5,0.5,2.0,0.00,1.0,18.4,2.5,0.00
3,1,2023-04-01 00:53:18,2023-04-01 01:01:28,1.0,1.50,1.0,N,249,79,1,10.0,3.5,0.5,1.0,0.00,1.0,16.0,2.5,0.00
4,2,2023-04-01 00:07:00,2023-04-01 00:17:16,2.0,1.49,1.0,N,158,246,1,11.4,1.0,0.5,1.0,0.00,1.0,17.4,2.5,0.00


In [ ]:
df_yellow_taxis.to_csv('taxis_yellow.csv')

In [ ]:
def yellow_taxis_transformation():
  for link in yellow_taxis[:1]:
    df = pd.read_parquet(link)
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'], unit='ms')
    df['tpep_pickup_date'] = df['tpep_pickup_datetime'].dt.date
    df['tpep_pickup_time'] = df['tpep_pickup_datetime'].dt.time
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'], unit='ms')
    df['tpep_dropoff_date'] = df['tpep_dropoff_datetime'].dt.date
    df['tpep_dropoff_time'] = df['tpep_dropoff_datetime'].dt.time

    df['VendorID'] = df['VendorID'].replace(1, 'Creative Mobile Technologies, LLC').replace(2, 'VeriFone Inc.')
    df['RatecodeID'] = df.RatecodeID.replace(1, 'Standard rate').replace(2, 'JFK').replace(3, 'Newark').replace(4, 'Nassau or Westchester').replace(5, 'Negotiated fare').replace(6, 'Group ride')
    df['store_and_fwd_flag'] = df['store_and_fwd_flag'].replace('Y', 'store and forward trip').replace('N', 'not a store and forward trip')
    df['payment_type'] = df['payment_type'].replace(1,'Credit card').replace(2, 'Cash').replace(3, 'No charge').replace(4,'Dispute').replace(5,'Unknown').replace(6,'Voided trip')

  return df


In [ ]:
yellow_taxis_transformation()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,tpep_pickup_date,tpep_pickup_time,tpep_dropoff_date,tpep_dropoff_time
0,"Creative Mobile Technologies, LLC",2023-04-01 00:14:49,2023-04-01 00:45:01,2.0,4.90,Standard rate,not a store and forward trip,48,223,Credit card,...,6.00,0.00,1.0,39.90,2.5,0.00,2023-04-01,00:14:49,2023-04-01,00:45:01
1,VeriFone Inc.,2023-04-01 00:00:24,2023-04-01 00:56:19,1.0,21.89,JFK,not a store and forward trip,132,43,Cash,...,0.00,6.55,1.0,81.80,2.5,1.25,2023-04-01,00:00:24,2023-04-01,00:56:19
2,"Creative Mobile Technologies, LLC",2023-04-01 00:03:50,2023-04-01 00:14:42,2.0,1.30,Standard rate,not a store and forward trip,148,113,Credit card,...,2.00,0.00,1.0,18.40,2.5,0.00,2023-04-01,00:03:50,2023-04-01,00:14:42
3,"Creative Mobile Technologies, LLC",2023-04-01 00:53:18,2023-04-01 01:01:28,1.0,1.50,Standard rate,not a store and forward trip,249,79,Credit card,...,1.00,0.00,1.0,16.00,2.5,0.00,2023-04-01,00:53:18,2023-04-01,01:01:28
4,VeriFone Inc.,2023-04-01 00:07:00,2023-04-01 00:17:16,2.0,1.49,Standard rate,not a store and forward trip,158,246,Credit card,...,1.00,0.00,1.0,17.40,2.5,0.00,2023-04-01,00:07:00,2023-04-01,00:17:16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3288245,VeriFone Inc.,2023-04-30 23:50:19,2023-05-01 00:06:11,NaN,2.02,NaN,None,246,79,0,...,3.08,0.00,1.0,18.49,NaN,NaN,2023-04-30,23:50:19,2023-05-01,00:06:11
3288246,VeriFone Inc.,2023-04-30 23:28:32,2023-04-30 23:38:32,NaN,2.36,NaN,None,114,68,0,...,0.00,0.00,1.0,17.19,NaN,NaN,2023-04-30,23:28:32,2023-04-30,23:38:32
3288247,VeriFone Inc.,2023-04-30 23:25:26,2023-04-30 23:44:16,NaN,4.64,NaN,None,137,74,0,...,0.00,0.00,1.0,21.90,NaN,NaN,2023-04-30,23:25:26,2023-04-30,23:44:16
3288248,"Creative Mobile Technologies, LLC",2023-04-30 23:11:59,2023-04-30 23:26:19,NaN,0.00,NaN,None,162,151,0,...,4.20,0.00,1.0,27.20,NaN,NaN,2023-04-30,23:11:59,2023-04-30,23:26:19


---

# For Hire Vehicles transformation

In [ ]:
def fhv_transformation():
  for link in for_hire_vehicles[:1]:
    df = pd.read_parquet(link)


In [ ]:
manhattan_zones = [  4,  12,  13,  24,  41,  42,  43,  45,  48,  50,  68,  74,  75,
        79,  87,  88,  90, 100, 103, 104, 105, 107, 113, 114, 116, 120,
       125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153,
       158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224,
       229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246,
       249, 261, 262, 263]

df = pd.read_parquet(for_hire_vehicles[0])
df['SR_Flag'] = df['SR_Flag'].fillna('non shared ride').replace('1', 'shared ride').replace(1, 'shared ride')
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], unit='ms')
df['pickup_date'] = df['pickup_datetime'].dt.date
df['pickup_time'] = df['pickup_datetime'].dt.time
df['dropOff_datetime'] = pd.to_datetime(df['dropOff_datetime'], unit='ms')
df['dropOff_date'] = df['dropOff_datetime'].dt.date
df['dropOff_time'] = df['dropOff_datetime'].dt.time
df = df[df.PUlocationID.isin(manhattan_zones) | df.DOlocationID.isin(manhattan_zones)].reset_index(drop=True)

df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,pickup_date,pickup_time,dropOff_date,dropOff_time
0,B00225,2023-06-01 00:36:46,2023-06-01 01:02:34,NaN,116.0,non shared ride,B03404,2023-06-01,00:36:46,2023-06-01,01:02:34
1,B00225,2023-06-01 00:22:12,2023-06-01 00:28:14,NaN,232.0,non shared ride,B00225,2023-06-01,00:22:12,2023-06-01,00:28:14
2,B00254,2023-06-01 00:42:24,2023-06-01 00:45:43,163.0,229.0,non shared ride,B00254,2023-06-01,00:42:24,2023-06-01,00:45:43
3,B00254,2023-06-01 00:16:50,2023-06-01 00:30:53,13.0,79.0,non shared ride,B03404,2023-06-01,00:16:50,2023-06-01,00:30:53
4,B00254,2023-06-01 00:32:57,2023-06-01 00:54:34,87.0,74.0,non shared ride,B00254,2023-06-01,00:32:57,2023-06-01,00:54:34
...,...,...,...,...,...,...,...,...,...,...,...
227847,B03380,2023-06-30 23:46:31,2023-07-01 00:06:54,170.0,158.0,non shared ride,B03404,2023-06-30,23:46:31,2023-07-01,00:06:54
227848,B03380,2023-06-30 23:21:05,2023-06-30 23:33:34,13.0,148.0,non shared ride,B03404,2023-06-30,23:21:05,2023-06-30,23:33:34
227849,B03380,2023-06-30 23:41:54,2023-07-01 00:16:36,125.0,37.0,non shared ride,B03404,2023-06-30,23:41:54,2023-07-01,00:16:36
227850,B03380,2023-06-30 23:25:02,2023-06-30 23:47:18,127.0,152.0,non shared ride,B03380,2023-06-30,23:25:02,2023-06-30,23:47:18


In [ ]:
columnas_a_eliminar = ['dispatching_base_num', 'dropOff_datetime', 'pickup_date', 'pickup_time','SR_Flag', 'Affiliated_base_number']
columnas_a_eliminar2= ['dropOff_date','dropOff_time']

In [ ]:
df = df.drop(columns=columnas_a_eliminar2)


In [ ]:
df

,pickup_datetime,PUlocationID,DOlocationID
0,2023-06-01 00:36:46,NaN,116.0
1,2023-06-01 00:22:12,NaN,232.0
2,2023-06-01 00:42:24,163.0,229.0
3,2023-06-01 00:16:50,13.0,79.0
4,2023-06-01 00:32:57,87.0,74.0
...,...,...,...
227847,2023-06-30 23:46:31,170.0,158.0
227848,2023-06-30 23:21:05,13.0,148.0
227849,2023-06-30 23:41:54,125.0,37.0
227850,2023-06-30 23:25:02,127.0,152.0


In [ ]:
#df['PUlocationID'] = df['PUlocationID'].fillna(0)
#df['DOlocationID'] = df['DOlocationID'].fillna(0)
df['pickup_datetime'].fillna(pd.NA, inplace=True)  # Rellenar con valor nulo especial
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], errors='coerce')

In [ ]:
duplicados = df.duplicated(subset=['pickup_datetime'], keep=False)  # keep=False muestra todos los duplicados
duplicados_df = df[duplicados]
duplicados_df

,pickup_datetime,PUlocationID,DOlocationID
10,2023-06-01 00:04:17,13.0,265.0
30,2023-06-01 00:46:39,NaN,238.0
35,2023-06-01 00:30:01,NaN,244.0
51,2023-06-01 00:04:17,161.0,37.0
63,2023-06-01 00:26:28,NaN,237.0
...,...,...,...
227813,2023-06-30 23:39:01,107.0,141.0
227814,2023-06-30 23:28:12,211.0,33.0
227830,2023-06-30 23:36:34,113.0,49.0
227840,2023-06-30 23:52:48,144.0,25.0


In [ ]:
columnas_para_verificar = ['pickup_datetime', 'PUlocationID', 'DOlocationID']
df_sin_duplicados = df.drop_duplicates(subset=columnas_para_verificar, keep='first')

In [ ]:
df_sin_duplicados.to_csv('taxisgrises.csv')

In [ ]:
df.to_csv('taxis_grises.csv')

In [ ]:
df.dtypes

pickup_datetime    datetime64[ns]
PUlocationID              float64
DOlocationID              float64
dtype: object

In [ ]:
null_counts = df.isnull().sum()
null_counts


pickup_datetime         0
PUlocationID       113725
DOlocationID          112
dtype: int64

In [ ]:
df

,dispatching_base_num,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,pickup_date,pickup_time,dropOff_date,dropOff_time
0,B00225,NaN,116.0,non shared ride,B03404,2023-06-01,00:36:46,2023-06-01,01:02:34
1,B00225,NaN,232.0,non shared ride,B00225,2023-06-01,00:22:12,2023-06-01,00:28:14
2,B00254,163.0,229.0,non shared ride,B00254,2023-06-01,00:42:24,2023-06-01,00:45:43
3,B00254,13.0,79.0,non shared ride,B03404,2023-06-01,00:16:50,2023-06-01,00:30:53
4,B00254,87.0,74.0,non shared ride,B00254,2023-06-01,00:32:57,2023-06-01,00:54:34
...,...,...,...,...,...,...,...,...,...
227847,B03380,170.0,158.0,non shared ride,B03404,2023-06-30,23:46:31,2023-07-01,00:06:54
227848,B03380,13.0,148.0,non shared ride,B03404,2023-06-30,23:21:05,2023-06-30,23:33:34
227849,B03380,125.0,37.0,non shared ride,B03404,2023-06-30,23:41:54,2023-07-01,00:16:36
227850,B03380,127.0,152.0,non shared ride,B03380,2023-06-30,23:25:02,2023-06-30,23:47:18


In [ ]:
df = pd.read_parquet(high_volume_for_hire_vehicles[-1])
df

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B02864,B02864,2020-01-01 00:40:13,2020-01-01 00:43:34,2020-01-01 00:45:34,2020-01-01 01:02:20,148,90,1.93,...,2.70,2.75,NaN,0.0,18.25,N,N,,N,N
1,HV0003,B02682,B02682,2020-01-01 00:42:31,2020-01-01 00:46:33,2020-01-01 00:47:50,2020-01-01 00:53:23,114,79,0.81,...,1.31,2.75,NaN,0.0,10.84,N,N,,N,N
2,HV0003,B02764,B02764,2020-01-01 00:01:42,2020-01-01 00:02:06,2020-01-01 00:04:37,2020-01-01 00:21:49,4,125,2.53,...,1.39,2.75,NaN,3.0,11.73,N,N,,N,N
3,HV0003,B02764,B02764,2020-01-01 00:21:23,2020-01-01 00:26:02,2020-01-01 00:26:36,2020-01-01 00:33:00,231,113,1.11,...,0.75,2.75,NaN,0.0,5.84,N,N,,N,N
4,HV0003,B02764,B02764,2020-01-01 00:32:20,2020-01-01 00:37:06,2020-01-01 00:37:49,2020-01-01 00:46:59,114,144,1.10,...,1.03,2.75,NaN,0.0,7.69,N,N,,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20569363,HV0005,B02510,None,2020-01-31 23:27:10,NaT,2020-01-31 23:32:57,2020-01-31 23:39:09,132,10,2.73,...,0.91,0.00,NaN,0.0,0.00,Y,N,N,N,N
20569364,HV0003,B02764,B02764,2020-01-31 23:23:50,2020-01-31 23:27:46,2020-01-31 23:29:14,2020-01-31 23:57:17,48,148,4.95,...,1.66,2.75,NaN,0.0,19.30,N,N,,N,N
20569365,HV0003,B02395,B02395,2020-01-31 22:57:49,2020-01-31 22:58:23,2020-01-31 23:01:04,2020-01-31 23:15:42,152,159,2.78,...,0.00,0.00,NaN,0.0,10.29,N,N,,N,N
20569366,HV0003,B02395,B02395,2020-01-31 23:14:16,2020-01-31 23:15:42,2020-01-31 23:17:42,2020-01-31 23:51:42,159,191,16.69,...,0.00,0.00,NaN,0.0,35.11,N,N,,N,N


In [ ]:
df.to_csv('taxis_negros.csv')

NameError: ignored

In [ ]:
df.columns

Index(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag'],
      dtype='object')

In [ ]:
df[['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID']].head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID
0,HV0003,B02864,B02864,2020-01-01 00:40:13,2020-01-01 00:43:34,2020-01-01 00:45:34,2020-01-01 01:02:20,148,90
1,HV0003,B02682,B02682,2020-01-01 00:42:31,2020-01-01 00:46:33,2020-01-01 00:47:50,2020-01-01 00:53:23,114,79
2,HV0003,B02764,B02764,2020-01-01 00:01:42,2020-01-01 00:02:06,2020-01-01 00:04:37,2020-01-01 00:21:49,4,125
3,HV0003,B02764,B02764,2020-01-01 00:21:23,2020-01-01 00:26:02,2020-01-01 00:26:36,2020-01-01 00:33:00,231,113
4,HV0003,B02764,B02764,2020-01-01 00:32:20,2020-01-01 00:37:06,2020-01-01 00:37:49,2020-01-01 00:46:59,114,144


In [ ]:
df[['trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee']].head()

,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee
0,1.93,1006,30.44,0.0,0.76,2.70,2.75,NaN
1,0.81,333,14.80,0.0,0.37,1.31,2.75,NaN
2,2.53,1032,15.63,0.0,0.47,1.39,2.75,NaN
3,1.11,384,8.44,0.0,0.21,0.75,2.75,NaN
4,1.10,550,11.57,0.0,0.29,1.03,2.75,NaN


In [ ]:
df[['tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag']].head()

,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,0.0,18.25,N,N,,N,N
1,0.0,10.84,N,N,,N,N
2,3.0,11.73,N,N,,N,N
3,0.0,5.84,N,N,,N,N
4,0.0,7.69,N,N,,N,N


# Transformation

In [ ]:
df = df[df.PULocationID.isin(manhattan_zones) | df.DOLocationID.isin(manhattan_zones)].reset_index(drop=True)
df['hvfhs_company'] = df['hvfhs_license_num']
df['hvfhs_company'] = df['hvfhs_company'].replace('HV0002','Juno').replace('HV0003','Uber').replace('HV0004', 'Via').replace('HV0005','Lyft')
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], unit='ms')
df['pickup_date'] = df['pickup_datetime'].dt.date
df['pickup_time'] = df['pickup_datetime'].dt.time
df['dropOff_datetime'] = pd.to_datetime(df['dropOff_datetime'], unit='ms')
df['dropOff_date'] = df['dropOff_datetime'].dt.date
df['dropOff_time'] = df['dropOff_datetime'].dt.time
df['shared_request_flag'] = df['shared_request_flag'].replace('Y', 'passenger accepted').replace('N', 'passenger did not accepted')
df['shared_match_flag'] = df['shared_match_flag'].replace('Y', 'shared trip').replace('N', 'non shared trip')


---

In [ ]:
import io
import pandas as pd
import requests
from bs4 import BeautifulSoup


if 'data_loader' not in globals():
    from mage_ai.data_preparation.decorators import data_loader
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test


@data_loader
def load_data_from_api(*args, **kwargs):
    """
    Template for loading data from API
    """
    url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
    response = requests.get(url)

    # Contenido de la pagina
    soup = BeautifulSoup(response.content, "html.parser")
    # Creo una lista para alojar todos los links que sean de los archivos parquet
    parquet_files = []

    # Hago una filtración de los archivos que pertenecen solamente a los años que nos importan
    selected_years = ['2020', '2021', '2022', '2023']
    years = [2020, 2021, 2022, 2023]

    # Busco todos los elementos que sean links
    for link in soup.find_all('a',href=True):
        if link['href'].endswith('.parquet'): # Si el elemento termina en .parquet, añado el elemento a la lista parquet_files
            if any(year in link['href'] for year in selected_years):
                print(link['href'])
                parquet_files.append(link['href'])

    high_volume_fhv = []
    for link in parquet_files:
        if 'fhvhv_tripdata' in link: # Todos los links que contengan 'yellow_tripdata' en su texto
            high_volume_fhv.append(link)

    high_volume_fhv = sorted(high_volume_fhv, reverse=True)

    # IdLocation de Manhattan
    manhattan_zones = [  4,  12,  13,  24,  41,  42,  43,  45,  48,  50,  68,  74,  75,
        79,  87,  88,  90, 100, 103, 104, 105, 107, 113, 114, 116, 120,
       125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153,
       158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224,
       229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246,
       249, 261, 262, 263]

    # Creo la lista para ir depositando todos los dataframes
    dataframes = []
    for parquet_file in high_volume_fhv:
        df = pd.read_parquet(parquet_file)
        df = df[['pickup_datetime','PULocationID','DOLocationID']]
        df = df[df['pickup_datetime'].dt.year.isin(years)].reset_index(drop=True)

        # Selecciono solamente los registros que pertenecen a viajes del distrito de Manhattan
        df = df[df.PULocationID.isin(manhattan_zones) & df.DOLocationID.isin(manhattan_zones)].reset_index(drop=True)
        dataframes.append(df)

    # Ahora concateno todos esos dataframes en uno solo
    full_df = pd.concat(dataframes, ignore_index=True)
    '''
    full_df['idblack'] = ('black' + full_df['pickup_datetime'].dt.year.astype(str) + full_df['pickup_datetime'].dt.strftime('%m%d%H%M%S'))

    full_df['idblack'] = full_df['idblack'].str.replace(':','')

    full_df = full_df[['idblack',
    'pickup_datetime',
    'PULocationID',
    'DOLocationID']]
    '''

    return full_df

@test
def test_output(output, *args) -> None:
    """
    Template code for testing the output of the block.
    """
    assert output is not None, 'The output is undefined'

---
## Limpieza y transformaciónes para Taxis Grises Prepandemia

In [ ]:
pd.read_parquet(for_hire_vehicles[0])

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2019-12-01 00:47:00,2019-12-01 00:52:00,264.0,264.0,None,B00009
1,B00009,2019-12-01 00:27:00,2019-12-01 00:44:00,264.0,264.0,None,B00009
2,B00014,2019-12-01 00:42:18,2019-12-01 01:31:31,264.0,264.0,None,B00014
3,B00014,2019-12-01 00:43:08,2019-12-01 01:07:38,264.0,264.0,None,B00014
4,B00021,2019-12-01 00:52:19,2019-12-01 00:59:39,56.0,56.0,None,B00021
...,...,...,...,...,...,...,...
2044191,B03266,2019-12-31 23:22:10,2019-12-31 23:30:08,7.0,7.0,None,B03266
2044192,B03266,2019-12-31 23:37:41,2019-12-31 23:45:39,179.0,179.0,None,B03266
2044193,B03266,2019-12-31 23:54:59,2020-01-01 00:02:57,193.0,193.0,None,B03266
2044194,B03266,2019-12-31 23:43:05,2019-12-31 23:51:03,179.0,179.0,None,B03266


In [ ]:
for_hire_vehicles = sorted(list(set(for_hire_vehicles)), reverse=True)

In [ ]:
# IdLocation de Manhattan
manhattan_zones = [  4,  12,  13,  24,  41,  42,  43,  45,  48,  50,  68,  74,  75,
    79,  87,  88,  90, 100, 103, 104, 105, 107, 113, 114, 116, 120,
    125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153,
    158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224,
    229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246,
    249, 261, 262, 263]

# Creo la lista para ir depositando todos los dataframes
dataframes = []
for parquet_file in for_hire_vehicles:
    # Carga los datos con el tipo de dato especificado
    df = pd.read_parquet(parquet_file, columns=['pickup_datetime','PUlocationID', 'DOlocationID'])
    print(parquet_file.split('/')[-1])
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
    df = df[df['pickup_datetime'].dt.year.isin(years)].reset_index(drop=True)

    # Selecciono solamente los registros que pertenecen a viajes del distrito de Manhattan
    df = df[df.PUlocationID.isin(manhattan_zones) & df.DOlocationID.isin(manhattan_zones)].reset_index(drop=True)
    dataframes.append(df)

# Ahora concateno todos esos dataframes en uno solo
full_df = pd.concat(dataframes, ignore_index=True)
full_df['idgrey'] = ('grey' + full_df['pickup_datetime'].dt.year.astype(str) + full_df['pickup_datetime'].dt.strftime('%m%d%H%M%S'))

full_df['idgrey'] = full_df['idgrey'].str.replace(':','')

full_df = full_df[['idgrey',
'pickup_datetime',
'PUlocationID',
'DOlocationID']]

fhv_tripdata_2019-12.parquet
fhv_tripdata_2019-11.parquet
fhv_tripdata_2019-10.parquet
fhv_tripdata_2019-09.parquet
fhv_tripdata_2019-08.parquet
fhv_tripdata_2019-07.parquet
fhv_tripdata_2019-06.parquet
fhv_tripdata_2019-05.parquet
fhv_tripdata_2019-04.parquet
fhv_tripdata_2019-03.parquet
fhv_tripdata_2019-02.parquet
fhv_tripdata_2019-01.parquet
fhv_tripdata_2018-12.parquet
fhv_tripdata_2018-11.parquet
fhv_tripdata_2018-10.parquet
fhv_tripdata_2018-09.parquet
fhv_tripdata_2018-08.parquet
fhv_tripdata_2018-07.parquet


KeyboardInterrupt: ignored

In [ ]:
import io
import pandas as pd
import requests
from bs4 import BeautifulSoup

if 'data_loader' not in globals():
    from mage_ai.data_preparation.decorators import data_loader
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test


@data_loader
def load_data_from_api(*args, **kwargs):
    """
    Template for loading data from API
    """
    url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
    response = requests.get(url)

    # Contenido de la pagina
    soup = BeautifulSoup(response.content, "html.parser")
    # Creo una lista para alojar todos los links que sean de los archivos parquet
    parquet_files = []

    # Hago una filtración de los archivos que pertenecen solamente a los años que nos importan

    selected_years = list(map(lambda x: str(x),range(2017,2020)))
    years = list(range(2017,2020))

    # Busco todos los elementos que sean links
    for link in soup.find_all('a',href=True):
        if link['href'].endswith('.parquet'): # Si el elemento termina en .parquet, añado el elemento a la lista parquet_files
            if any(year in link['href'] for year in selected_years):
                print(link['href'])
                parquet_files.append(link['href'])

    for_hire_vehicles = []
    for link in parquet_files:
        if 'fhv_tripdata' in link: # Todos los links que contengan 'yellow_tripdata' en su texto
            for_hire_vehicles.append(link)

    for_hire_vehicles = sorted(for_hire_vehicles, reverse=True)

    # IdLocation de Manhattan
    manhattan_zones = [  4,  12,  13,  24,  41,  42,  43,  45,  48,  50,  68,  74,  75,
        79,  87,  88,  90, 100, 103, 104, 105, 107, 113, 114, 116, 120,
       125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153,
       158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224,
       229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246,
       249, 261, 262, 263]

    dataframes = []
    for parquet_file in for_hire_vehicles:
        # Carga los datos con el tipo de dato especificado
        df = pd.read_parquet(parquet_file, columns=['pickup_datetime','PUlocationID', 'DOlocationID'])
        df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
        df = df[df['pickup_datetime'].dt.year.isin(years)].reset_index(drop=True)

        # Selecciono solamente los registros que pertenecen a viajes del distrito de Manhattan
        df = df[df.PUlocationID.isin(manhattan_zones) & df.DOlocationID.isin(manhattan_zones)].reset_index(drop=True)
        dataframes.append(df)

    # Ahora concateno todos esos dataframes en uno solo
    full_df = pd.concat(dataframes, ignore_index=True)
    full_df['idgrey'] = ('grey' + full_df['pickup_datetime'].dt.year.astype(str) + full_df['pickup_datetime'].dt.strftime('%m%d%H%M%S'))

    full_df['idgrey'] = full_df['idgrey'].str.replace(':','')

    full_df = full_df[['idgrey',
    'pickup_datetime',
    'PUlocationID',
    'DOlocationID']]

    return full_df

@test
def test_output(output, *args) -> None:
    """
    Template code for testing the output of the block.
    """
    assert output is not None, 'The output is undefined'

,pickup_datetime,PUlocationID,DOlocationID
0,2019-09-01 00:35:00,264.0,264.0
1,2019-09-01 00:48:00,264.0,264.0
2,2019-09-01 00:16:18,264.0,264.0
3,2019-09-01 00:55:03,264.0,264.0
4,2019-09-01 00:13:08,264.0,264.0
...,...,...,...
1248515,2019-09-30 23:29:16,264.0,265.0
1248516,2019-09-30 23:50:43,264.0,167.0
1248517,2019-09-30 23:07:02,NaN,177.0
1248518,2019-09-30 23:19:41,NaN,89.0


In [ ]:
argon2:$argon2id$v=19$m=10240,t=10,p=8$UtDBGu6C7gTHhHUVPvlS3g$dGjaxwaH+r4ylpINgDrKbv7MbyKMINNCtwtDedSFFLQ